In [45]:
import shutil

In [ ]:
## example output upload
!aws s3 cp images/test s3://tickercardiology-echocv-sagemaker/model-data/test --recursive
!aws s3 cp images/train s3://tickercardiology-echocv-sagemaker/model-data/train --recursive

In [1]:
!python sagemaker_training_job.py

No handlers could be found for logger "sagemaker"

Training ...


In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlowModel

bucket = "sagemaker-ap-southeast-2-611188727347"
key = "sagemaker-tensorflow-2020-10-29-05-02-56-236/output/model.tar.gz"
model_path =  "s3://{}/{}".format(bucket, key)

model = TensorFlowModel(model_data=model_path, role=sagemaker.get_execution_role(), entry_point = 'train.py')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.2xlarge')

In [11]:
from tensorflow.python.keras.preprocessing.image import load_img
import numpy as np
test_image = load_img("images/test/a2c/a2c-1_00.jpg", target_size=(224, 224))
test_image_array = np.array(test_image).reshape((1, 224, 224, 3))
result = predictor.predict({'inputs_input': test_image_array})

print(result)

{'outputs': {u'dense_2': {'dtype': 1, 'float_val': [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'tensor_shape': {'dim': [{'size': 1L}, {'size': 8L}]}}}, 'model_spec': {'signature_name': u'serving_default', 'version': {'value': 1603948354L}, 'name': u'generic_model'}}


In [ ]:
#######################

In [6]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [7]:
print(region)

ap-southeast-2


In [8]:
bucket = "tickercardiology-echocv-sagemaker"
key = "model-data"
training_data_uri = "s3://{}/{}/train/".format(bucket, key)
# training_data_uri = 's3://sagemaker-sample-data-{}/tensorflow/mnist'.format(region)

In [18]:
!pygmentize 'keras_model_fn.py'

import os
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.losses import categorical_crossentropy  

import numpy as np

INPUT_TENSOR_NAME = "inputs_input" # Watch out, it needs to match the name of the first layer + "_input"
BATCH_SIZE = 64
HEIGHT = 224
WIDTH = 224
DEPTH = 3

def model(x_train, y_train, x_test, y_test):
    model = Sequential()
    model.add(Conv2D(input_shape=(HEIGHT,WIDTH,DEPTH),filters=64,kernel_size=(3,3),padding="same",activation="relu",name="inputs"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(fil

In [21]:
estimator2 = TensorFlow(entry_point='keras_model_fn.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type='ml.m5.2xlarge',
                             framework_version='2.1.0',
                             py_version='py2',
                             distributions={'parameter_server': {'enabled': True}})

In [ ]:
estimator2.fit(training_data_uri)

2020-11-02 06:50:53 Starting - Starting the training job...
2020-11-02 06:50:55 Starting - Launching requested ML instances......
2020-11-02 06:51:59 Starting - Preparing the instances for training...
2020-11-02 06:52:52 Downloading - Downloading input data
2020-11-02 06:52:52 Training - Downloading the training image..
2020-11-02 06:53:06 Training - Training image download completed. Training in progress.2020-11-02 06:53:08,841 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-11-02 06:53:08,850 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-11-02 06:53:09,333 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-11-02 06:53:10,764 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-11-02 06:53:10,782 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-11-02 06:53:10,792 sagemaker-containers INFO     Invoking user script